# 抖音爬取视频

## 获取单个去水印视频

In [5]:
import requests as req
import requests
import re
import json

requests.packages.urllib3.disable_warnings()
headers = {
    "User-Agent": "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)"
}


def get_location(original_url):
    response = req.get(original_url,
                       headers=headers,
                       verify=False,
                       allow_redirects=False)
    return response.headers['Location']


def parse_single_url(original_url):
    long_url = get_location(original_url)
    regex = 'video/(\d+)?'
    return re.findall(regex, long_url)[0]


def parse_result(key):
    api_url = 'https://www.iesdouyin.com/web/api/v2/aweme/iteminfo/?item_ids=' + key
    response = req.get(api_url, headers=headers, params=None, verify=False)
    result = json.loads(response.text)['item_list'][0]

    nwm_video_url = str(result['video']['play_addr']['url_list'][0]).replace(
        'playwm', 'play')
    return get_location(nwm_video_url)

def get_http_txt(txt):
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')    # 匹配模式
    # string = '2.05 eOK:/ 复制打开抖音，看看【企鹅冷知识的作品】 https://v.douyin.com/6FQv8ER/'
    string = txt
    url = re.findall(pattern,string)
    return url[0]

def download_file(download_url, file_path):
    response = req.get(download_url, verify=False, stream=True)
    chunk_size = 1024
    if response.status_code == 200:
        with open(file_path, 'wb') as file:
            for data in response.iter_content(chunk_size=chunk_size):
                file.write(data)

if __name__ == "__main__":
    # key = parse_single_url('https://v.douyin.com/2qsMWfy/')
    http_txt= get_http_txt('2.05 eOK:/ 复制打开抖音，看看【企鹅冷知识的作品】 https://v.douyin.com/6FQv8ER/')
    print(http_txt)
    # key = parse_single_url('https://v.douyin.com/6FQv8ER/')
    key = parse_single_url(http_txt)
    url = parse_result(key)
    download_file(url, './红婆视频2.mp4')

https://v.douyin.com/6FQv8ER/


In [ ]:
http_txt = get_http_txt(
    '9.【技巧生活的个:/ 人主页】长按复制此条消息，长按复制打开抖音搜索，查看TA的更多作品##XzHKVruTuv8##[抖音口令]')


## 批量去抖音视频水印

In [24]:
import requests as req
import re
import json

req.packages.urllib3.disable_warnings()


headers = {
    "User-Agent": 'Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)"'
}

def get_sec_uid(original_url):
    response = req.get(original_url, headers=headers, verify=False, allow_redirects=False)
    long_url = response.headers['Location']
    sec_uid = re.findall('(?<=sec_uid=)[a-z，A-Z，0-9, _, -]+', long_url, re.M | re.I)[0]
    return sec_uid


def get_home_items(sec_uid, max_cursor=0, urls=[]):
    params = {
        'sec_uid': sec_uid,
        'count': 21,
        'max_cursor': max_cursor,
        'reflow_source': 'reflow_page'
    }
    aweme_api = 'https://www.iesdouyin.com/web/api/v2/aweme/post/?'
    response = req.get(aweme_api, headers=headers, params=params, verify=False)
    data = json.loads(response.text)
    list = data['aweme_list']
    aweme_list_len = len(list)
    for i in range(aweme_list_len):
        title = re.sub('[\/:*?"<>|]', '-', list[i]['desc'])
        url = list[i]['video']['play_addr']['url_list'][0]
        # print(title + ":" + url)
        ext = '.mp3' if list[i]['video']['ratio'] == 'default' else '.mp4'
        item = (title, url, ext)
        urls.append(item)
    if data['has_more']:
        return get_home_items(sec_uid, data['max_cursor'], urls)
    return urls

def download_file(download_url, file_path):
    response = req.get(download_url, verify=False, stream=True)
    chunk_size = 1024
    if response.status_code == 200:
        with open(file_path, 'wb') as file:
            for data in response.iter_content(chunk_size=chunk_size):
                file.write(data)

if __name__ == "__main__":
    # 先获取原始地址
    sec_uid = get_sec_uid('https://v.douyin.com/2pa4REG/')
    # sec_uid = 'MS4wLjABAAAAjU-tAMdEIb2lAic3sb8Wch7e85XMKPLq8jZiVnIVNlA'
    list = get_home_items(sec_uid)
    for key,val in enumerate(list):
        title = val[0]
        url = val[1]
        ext = val[2]
        # print(key)
        print(title,url,ext)
        
        if key == 9:
            break;
        download_file(url, f'./movies/{key}{ext}')
        print(f'{title} ------- 下载完成')

 https://sf3-cdn-tos.douyinstatic.com/obj/ies-music/7120967683574614815.mp3 .mp3
 ------- 下载完成
 https://sf3-cdn-tos.douyinstatic.com/obj/ies-music/7120967683574614815.mp3 .mp3
 ------- 下载完成
 https://sf3-cdn-tos.douyinstatic.com/obj/ies-music/7120967683574614815.mp3 .mp3
 ------- 下载完成
 https://sf3-cdn-tos.douyinstatic.com/obj/ies-music/7120967683574614815.mp3 .mp3
 ------- 下载完成
 https://sf3-cdn-tos.douyinstatic.com/obj/ies-music/7120967683574614815.mp3 .mp3
 ------- 下载完成
 https://sf3-cdn-tos.douyinstatic.com/obj/ies-music/7120967683574614815.mp3 .mp3
 ------- 下载完成
 https://sf3-cdn-tos.douyinstatic.com/obj/ies-music/7120967683574614815.mp3 .mp3
 ------- 下载完成
 https://sf3-cdn-tos.douyinstatic.com/obj/ies-music/7120967683574614815.mp3 .mp3
 ------- 下载完成
 https://sf3-cdn-tos.douyinstatic.com/obj/ies-music/7120967683574614815.mp3 .mp3
 ------- 下载完成
 https://sf3-cdn-tos.douyinstatic.com/obj/ies-music/7120967683574614815.mp3 .mp3
